use of 'package:http/http.dart' as http;


```dart
import 'dart:convert';

import 'package:http/http.dart' as http;
import 'package:toontrail/models/webtoon_model.dart';

class ApiService {
  final String baseUrl = "https://webtoon-crawler.nomadcoders.workers.dev";
  final String today = "today";

  void getTodaysToons() async {
    final url = Uri.parse('$baseUrl/$today');
    final response = await http.get(url);
    if (response.statusCode == 200) {
      final List<dynamic> webtoons = jsonDecode(response.body);
      for (var webtoon in webtoons) {
        WebtoonModel.fromJson(webtoon);
      }
      return;
    }
    throw Error();
  }
}
```

## Two ways of calling Future data
#### 1. basic
  * Write an async-await function that waits on the fetched data, refresh(setState(() {})) the screen.(waitForWebtoons)
  * put the results in the State(webtoons, isLoading)
  * Use initState() function: allows to call the fucntion
```dart
  List<WebtoonModel> webtoons = [];
  bool isLoading = true;

  void waitForWebtoons() async {
    webtoons = await ApiService.getTodaysToons();
    isLoading = false;
    setState(() {});
  }

  @override
  void initState() {
    super.initState();
    waitForWebtoons();
  }
```

- manual, exposed to mistakes
- this is using State, but we might wanna avoid using State
#### 2. more advanced(better ways)
  * You can still fetch data with Stateless widget
  * Use FutureBuilder widget
```dart
//Calling the service as a future variable to the page first
Future<List<WebtoonModel>> webtoons = ApiService.getTodaysToons();

  @override
  Widget build(BuildContext context) {
    return Scaffold(
        backgroundColor: Colors.white,
        appBar: AppBar(
          foregroundColor: Colors.green,
          backgroundColor: Colors.white,
          elevation: 2,
          title: const Text(
            "Today's webtoons",
            style: TextStyle(
              fontSize: 24,
              fontWeight: FontWeight.w600,
            ),
          ),
        ),
        //Use the service(webtoon) at the future builder
        body: FutureBuilder(
          future: webtoons,
          builder: (context, snapshot) {
            if (snapshot.hasData) {
              return const Text("There is Data.");
            }
            return const Text("Loading...");
          },
        ));
  }
```
 * FutureBuilder widget will wait for the future data without explicitly use await function.

ListView Widget: It is used to show many consecutive items. optimized to list items.
ListView.builder(
    itemBuilder: instead of creating items all at once, it applies itembuilder function to the items, it will give you the index of the item. it can access the index of items.
)

``**Navigator.push**`` gives a nice animation to render a new widget screen.

### Hero widget
Use hero widget on the two separated screens and give the same tag on each screen

### Getting Future data in stateful widget

```dart
//webtoon_widget.dart
  @override
  Widget build(BuildContext context) {
    return GestureDetector(
      onTap: () {
        Navigator.push(
          context,
          MaterialPageRoute(
            builder: (context) => DetailScreen(
              title: title,
              thumb: thumb,
              id: id,
            ),
            fullscreenDialog: true,
          ),
        );
      },
      ...
    ),
  }
```
* When the build function in stateless widget in the webtoon_widget.dart sends the title, thumb, and id to DetailScreen class, `it has been sent to the widget but not to the state in the widget yet.`
* State can grab the data by using `widget.data(title, thumb, id)` but cannot use the variable by itself like title, thumb, and id. 
```dart
class DetailScreen extends StatefulWidget {
  final String title, thumb, id;
    // This is the data sent from home screen when a poster is clicked.
  const DetailScreen({
    super.key,
    required this.title,
    required this.thumb,
    required this.id,
  });
    @override
  State<DetailScreen> createState() => _DetailScreenState();
}

    class _DetailScreenState extends State<DetailScreen> {
      late Future<WebtoonDetailModel> webtoon;
      late Future<List<WebtoonEpisodeModel>> episodes;

      @override
      void initState() {
        super.initState();
        webtoon = ApiService.getToonById(widget.id);
        episodes = ApiService.getLatestEpisodeById(widget.id);
      }
    ...
}
    
```

### Why is Stateful widget is used for one page and not on the other page?
* For home screen, the future data did not require arguments.
* For detail screen, on the other hand, the future data required an argument, id. 
* In the case of detail screen, the data cannot be loaded in the way that home screen calls it. This is because the argument has to be loaded ahead of the data fetching. 
* As a stateful widget separates the widget from the state, and the data has been sent to the widget, the stateful widget utilize the separation between the state and the widget for the usage of fetched data.

### setting state in a function even though you call the function initState:

```dart

class _DetailScreenState extends State<DetailScreen> {
  late Future<WebtoonDetailModel> webtoon;
  late Future<List<WebtoonEpisodeModel>> episodes;
  late SharedPreferences prefs;
  bool isLiked = false;

  Future initPrefs() async {
    prefs = await SharedPreferences.getInstance();
    final likedToons = prefs.getStringList('likedToons');
    if (likedToons != null) {
      if (likedToons.contains(widget.id) == true) {
        // Setting State is needed for UI refreash
        setState(() {
          isLiked = true;
        });
      }
    } else {
      await prefs.setStringList('likedToons', []);
    }
  }

  @override
  void initState() {
    super.initState();
    webtoon = ApiService.getToonById(widget.id);
    episodes = ApiService.getLatestEpisodeById(widget.id);
    initPrefs();
  } // initPrefs() function is called in initState() function but the function still needs setState() for reflection of the change.

  onHeartTap() async {
    final likedToons = prefs.getStringList('likedToons');
    if (likedToons != null) {
      if (isLiked) {
        likedToons.remove(widget.id);
      } else {
        likedToons.add(widget.id);
      }
      await prefs.setStringList('likedToons', likedToons);
      setState(() {
        isLiked = !isLiked;
      });
    }
  }
```
* **Initialization and State Changes**:
  
  * In `initState` method, you initialize some variables and call `initPrefs`, which sets up shared preferences and checks if a particular item is liked.

  * Inside `initPrefs`, if the item is already liked (i.e., found in the shared preferences), you call `setState` to update the `isLiked` variable. 
  - This triggers a rebuild of the widget, allowing the UI to reflect the change (e.g., showing or hiding a "liked" icon).

* **Why `setState` is Necessary**:
  
  * **Without `setState`**: If you change the value of `isLiked` directly within `initPrefs` without calling `setState`, the UI will not automatically update to reflect this change. The widget tree remains unaware of the update, so it will continue to show the previous state.
  * **With `setState`**: When you call `setState`, Flutter marks the widget as dirty and schedules a rebuild. During this rebuild, Flutter applies the new state to the widget, updating the UI to reflect any changes.
